In [1]:
import sys
sys.path.append('/kaggle/input/essentials')

In [2]:
!pip install guided-filter-pytorch

In [3]:
from sklearn.svm import SVC
from stream1 import FirstStream
from stream2 import SecondStream
from TwoStreamNet import TwoStreamNet
from img_converter import createHighFrequencyComponent, createLowFrequencyComponent
import torch
import numpy as np
import pandas as pd
import cv2
from torchvision.datasets import ImageFolder
from torchvision import transforms
from tqdm import tqdm
from pathlib import Path

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
 33%|███▎      | 77.1M/233M [00:00<00:00, 169MB/s] 

In [4]:
path = Path()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device being used: {device}")

# create the stream1 and stream2 model objects and load model file
stream_1 = FirstStream()
stream_2 = SecondStream()
stream_1.load_state_dict(torch.load('/kaggle/input/stream-models/stream1_model.pth',
                                    map_location=device
                                    ), 
                         strict=False)
stream_2.load_state_dict(torch.load('/kaggle/input/stream-models/stream2_model.pth', 
                                    map_location=device
                                    ), 
                         strict=False)

classifier = TwoStreamNet(stream_1, stream_2).to(device)
classifier.load_state_dict(torch.load('/kaggle/input/stream-models/joint_model.pth', 
                                      map_location=device
                                      ), 
                           strict=False)

device being used: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [5]:
# create the dataloader
BATCH_SIZE = 1
data_root = '/kaggle/input/11k-hands-training-dataset/content/drive/MyDrive/train_images/train'
#test_root = Path("G:/11k_hands/dataset/test")
vectors = []
labels = []

# override the ImageFolder to include the custom function
class CustomImageFolder(ImageFolder):
    def __init__(self, root, transform=None):
        super().__init__(root=root, transform=None)  # disable transform for now
        self.base_transform = transform  # keep the transform pipeline without the custom fn

    def __getitem__(self, index):
        path, target = self.samples[index]

        blurred_img = createLowFrequencyComponent(path)
        blurred_img = (blurred_img - blurred_img.min())/(blurred_img.max() - blurred_img.min())
        blurred_img = torch.from_numpy(blurred_img).permute(2, 0, 1).float()

        detailed_img = createHighFrequencyComponent(path)
        detailed_img = cv2.resize(detailed_img, (224, 224))
        detailed_img = np.expand_dims(detailed_img, axis=0)  # shape: (1, 224, 224)
        detailed_img = torch.from_numpy(detailed_img).float()
        
        if self.base_transform is not None:
            blurred_img = self.base_transform(blurred_img)
            detailed_img = self.base_transform(detailed_img)

        return blurred_img, detailed_img, target

base_transform = transforms.Compose([
    transforms.Resize((224, 224))
])

dataset = CustomImageFolder(root=data_root, transform=base_transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [6]:
#store each of the vectors to a csv file
classifier.eval()
loop = tqdm(dataloader, total=len(dataloader))

for (blurred_img, detailed_img, label) in loop:
    blurred_img, detailed_img, label = blurred_img.to(device), detailed_img.to(device), label.to(device)

    f1, f2, x = classifier(blurred_img, detailed_img)
    concatenated_feature = torch.concat([f1, f2], dim=1)
    concatenated_feature = torch.concat([concatenated_feature, x], dim=1)
    
    vectors.append(concatenated_feature.cpu().detach().numpy())
    labels.append(label.cpu().detach().numpy())

# save the features and labels to a file
vectors = np.array(vectors)
vectors = vectors.squeeze(axis=1)
df = pd.DataFrame(vectors)
df['Label'] = labels

df.to_csv('/kaggle/working/features.csv', index=False)

100%|██████████| 7754/7754 [1:57:17<00:00,  1.10it/s]


ValueError: Must pass 2-d input. shape=(7754, 1, 2124)